In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [20]:
!curl 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet' --output yellow_tripdata_202201.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 36.3M  100 36.3M    0     0  6227k      0  0:00:05  0:00:05 --:--:-- 5475k


In [21]:
!curl 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet' --output yellow_tripdata_202202.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.5M  100 43.5M    0     0  5876k      0  0:00:07  0:00:07 --:--:-- 5610k


In [3]:
df_jan = pd.read_parquet('yellow_tripdata_202201.parquet')

In [25]:
df_jan.describe

<bound method NDFrame.describe of          VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count   
0               1  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0  \
1               1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0   
2               2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0   
3               2  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0   
4               2  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0   
...           ...                  ...                   ...              ...   
2463926         2  2022-01-31 23:36:53   2022-01-31 23:42:51              NaN   
2463927         2  2022-01-31 23:44:22   2022-01-31 23:55:01              NaN   
2463928         2  2022-01-31 23:39:00   2022-01-31 23:50:00              NaN   
2463929         2  2022-01-31 23:36:42   2022-01-31 23:48:45              NaN   
2463930         2  2022-01-31 23:46:00   2022-02-01 00:13:00              N

In [19]:
len(df_jan.columns)

20

In [4]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']

In [38]:
# td = df_jan.duration.iloc[0]
df_jan.iloc[0]

VendorID                                   1
tpep_pickup_datetime     2022-01-01 00:35:40
tpep_dropoff_datetime    2022-01-01 00:53:29
passenger_count                          2.0
trip_distance                            3.8
RatecodeID                               1.0
store_and_fwd_flag                         N
PULocationID                             142
DOLocationID                             236
payment_type                               1
fare_amount                             14.5
extra                                    3.0
mta_tax                                  0.5
tip_amount                              3.65
tolls_amount                             0.0
improvement_surcharge                    0.3
total_amount                           21.95
congestion_surcharge                     2.5
airport_fee                              0.0
duration                     0 days 00:17:49
Name: 0, dtype: object

In [29]:
removed = df_jan[df_jan['duration']>timede]

17.816666666666666

In [6]:
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [5]:
df_jan['duration_seconds'] = df_jan.duration.apply(lambda x : x.total_seconds()/60)

In [6]:
df_jan.duration_seconds.describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration_seconds, dtype: float64

In [7]:
df = df_jan[(df_jan.duration_seconds >= 1) & (df_jan.duration_seconds <= 60)]

In [10]:
df.shape

(2421440, 21)

In [45]:
df_jan.shape

(2463931, 21)

In [46]:
2421440/ 2463931

0.9827547930522406

In [8]:
from sklearn.feature_extraction import DictVectorizer

In [9]:
dv = DictVectorizer()

In [10]:
categorical = ['DOLocationID','PULocationID']
numerical = ['trip_distance']

In [11]:
df[categorical] = df[categorical].astype(str)

/tmp/ipykernel_4002/1767203464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [12]:
train_dicts = df[categorical].to_dict(orient='records')

In [16]:
train_dicts

[{'DOLocationID': '236', 'PULocationID': '142'},
 {'DOLocationID': '42', 'PULocationID': '236'},
 {'DOLocationID': '166', 'PULocationID': '166'},
 {'DOLocationID': '68', 'PULocationID': '114'},
 {'DOLocationID': '163', 'PULocationID': '68'},
 {'DOLocationID': '161', 'PULocationID': '138'},
 {'DOLocationID': '87', 'PULocationID': '233'},
 {'DOLocationID': '152', 'PULocationID': '238'},
 {'DOLocationID': '236', 'PULocationID': '166'},
 {'DOLocationID': '141', 'PULocationID': '236'},
 {'DOLocationID': '229', 'PULocationID': '141'},
 {'DOLocationID': '90', 'PULocationID': '114'},
 {'DOLocationID': '113', 'PULocationID': '234'},
 {'DOLocationID': '79', 'PULocationID': '246'},
 {'DOLocationID': '140', 'PULocationID': '43'},
 {'DOLocationID': '151', 'PULocationID': '239'},
 {'DOLocationID': '141', 'PULocationID': '148'},
 {'DOLocationID': '107', 'PULocationID': '237'},
 {'DOLocationID': '7', 'PULocationID': '7'},
 {'DOLocationID': '263', 'PULocationID': '107'},
 {'DOLocationID': '107', 'PULoc

In [13]:
X_train = dv.fit_transform(train_dicts)

In [14]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [15]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration',
       'duration_seconds'],
      dtype='object')

In [16]:
target = 'duration_seconds'

In [17]:
y_train = df[target].values

In [73]:
y_train

array([17.81666667,  8.4       ,  8.96666667, ..., 11.        ,
       12.05      , 27.        ])

In [18]:
X_train

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842880 stored elements in Compressed Sparse Row format>

In [19]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [20]:
y_pred = lr.predict(X_train)

In [21]:
from sklearn.metrics import mean_squared_error

In [61]:
mean_squared_error(y_train, y_pred, squared=False)

6.986190742248472

In [52]:
df_feb = pd.read_parquet('yellow_tripdata_202202.parquet')

In [53]:
df_feb.duration = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb['duration_seconds'] = df_jan.duration.apply(lambda x : x.total_seconds()/60)

/tmp/ipykernel_4002/2966308775.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_feb.duration = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime


In [54]:
df_feb[categorical] = df_feb[categorical].astype(str)

In [55]:
df_feb = df_feb[(df_feb.duration_seconds >= 1) & (df_feb.duration_seconds <= 60)]

In [56]:
test_dicts = df_feb[categorical].to_dict(orient='records')

In [57]:
X_test = dv.transform(test_dicts)

In [58]:
X_test

<2421440x515 sparse matrix of type '<class 'numpy.float64'>'
	with 4842873 stored elements in Compressed Sparse Row format>

In [59]:
y_test = df_feb['duration_seconds'].values

In [60]:
y_test_preds = lr.predict(X_test)

In [62]:
mean_squared_error(y_test, y_test_preds, squared=False)

10.532424397747896